# Metaphor algorithm

1. Parse file and list objects & mapto implied by subsets, common structures/covers
2. Match structures  
    a. Match types (how to deal with mappable ones? eg. segment->linear)  
    b. Match affected and covers
3. Match representations (keeping the structure consistent)



In [9]:
import yaml
import re
import enum
from pprint import pprint
import itertools
import math

def get_spec(file_path):
  with open(file_path, 'r') as file_handle:
    spec = yaml.safe_load(file_handle)
  return spec

In [10]:
ex_spec = get_spec('video-editor.yaml')

In [11]:
def process_listable(target, func):
  if target is None:
    return

  if type(target) == list:
    for target_item in target:
      func(target_item)
  else:
    func(target)

## Parsing for objects

In [12]:
def strip_type(obj):
  return re.sub(r'\((\w|-)*\) ', '', obj)

def register_object(registry, obj):
  # -> means mapto
  # . means subset
  # / means component  # TODO: actually do this!!
  assert(type(obj) is str)
  obj = strip_type(obj)

  arrow_array = obj.split('->')
  for arrow_idx, arrow_term in enumerate(arrow_array):
    
    dot_array = arrow_term.split('.')
    for dot_idx, dot_term in enumerate(dot_array):
      subject = '.'.join(dot_array[:dot_idx + 1]) # join up to idx
      if not registry.get(subject):
        registry[subject] = set()
      
      if dot_idx > 0:
        # every sequence maps to the previous element eg. a.b.c => {a.b.c: {a.b}, a.b: {a}}
        previous = '.'.join(dot_array[:dot_idx])
        registry[subject].add(previous)
    
    if arrow_idx > 0:
      # a pair of arrows lhs->rhs => {lhs: {rhs}}
      lhs = registry[arrow_array[arrow_idx - 1]]
      rhs = arrow_array[arrow_idx]
      lhs.add(rhs)

In [13]:
# Tests
print('testing: a')
test = {}
print('expected:')
pprint({'a': set()})
print('got:')
register_object(test, 'a')
pprint(test)

print()

print('testing: a.b.c')
test = {}
print('expected:')
pprint({'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}})
print('got:')
register_object(test, 'a.b.c')
pprint(test)

print()

print('testing: a->b->c')
test = {}
print('expected:')
pprint({'a': {'b'}, 'b': {'c'}, 'c': set()})
print('got:')
register_object(test, 'a->b->c')
pprint(test)

print()

print('testing: a.b->x->z.w')

test = {}
register_object(test, 'a.b->x->z.w')
print('expected:')
print({'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}})
print('got:')
pprint(test)


testing: a
expected:
{'a': set()}
got:
{'a': set()}

testing: a.b.c
expected:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}
got:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}

testing: a->b->c
expected:
{'a': {'b'}, 'b': {'c'}, 'c': set()}
got:
{'a': {'b'}, 'b': {'c'}, 'c': set()}

testing: a.b->x->z.w
expected:
{'a': set(), 'a.b': {'a', 'x'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}
got:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}


In [14]:
def register_struct_objects(struct, registry, process_func):
  if struct.get('type') == 'group':
    # Groups also behave as objects, so register them
    name = struct.get('name')
    if name is None:
      print('Warning! No name provided for group. Using `NO NAME PROVIDED` instead. TODO: generate ID.')
      name = 'NO NAME PROVIDED'

    if registry.get(name) is None:
      registry[name] = set()
    
    # TODO: do groups map to their elements? Really, does the transitive property apply? My hunch is no, but need to think more

  process_listable(struct.get('affects'), process_func)
  process_listable(struct.get('covered-by'), process_func)
  # TODO: relate all of the objects affected/covered by a structure. 

  for derivative in struct.get('structures', []):
    register_struct_objects(derivative, registry=registry, process_func=process_func)


def register_repr_objects(repr, registry, process_func):
  repr_type = repr.get('type', '')
  # print(repr_type)
  for repr_item in repr.get('objects', []):
    assert(len(repr_item.values()) == 1)
    repr_obj, target_objs = list(repr_item.items())[0]

    process_listable(target_objs, process_func)

    # Map every item to the associated representational object
    # eg. {message: {textbox}, author: {textbox}}
    def register_repr_maps(target):
      # note: this is a bit backwards compared to the syntax!
      if registry.get(target) is None:
        registry[target] = set()
      registry[target].add(repr_obj)
      # objects[target].add(repr_type + '/' + repr_obj)  # when we prefix the core stuff
    process_listable(target_objs, register_repr_maps)



def register_spec_objects(spec, registry=None): # {object: [mapto-targets]}
  if registry is None:
    registry = {}

  # TODO: deal with `objects` block
  def register_object_here(target):
    register_object(registry, target)
  
  process_listable(spec.get('objects'), register_object_here)

  for struct in spec.get('structures', []):
    register_struct_objects(struct, registry, register_object_here)
    
  for repr in spec.get('representations', []):
    register_repr_objects(repr, registry, register_object_here)
  
  return registry

In [15]:
pprint(register_spec_objects(ex_spec))

{'playhead': {'vlines', 'videos.in-editor/images'},
 'playhead->videos.in-editor/images': {'rects'},
 'timestamps': {'vlines'},
 'tracks': {'regions'},
 'videos': set(),
 'videos.in-editor': {'rects', 'videos'},
 'videos.in-editor/images': {'videos'},
 'videos/first-frame': {'regions'}}


In [16]:
core_spec = get_spec('core.yaml')
core_objs = {}

# TODO: probably need to consider prefixing gui on these
for repr_type in core_spec.get('representation-types', []):
  # print(repr_type['name'])
  register_spec_objects(repr_type, core_objs)

# register_spec_objects(core_spec, core_objs)

pprint(core_objs)

{'hlines': {'lines'},
 'icons': {'regions'},
 'lines': {'regions'},
 'points': set(),
 'rects': {'regions'},
 'regions': set(),
 'vlines': {'lines'}}


In [17]:
def get_node_join(left, right, relations, visited=set()):
  # TODO: I bet there's 
  assert(type(left) is str)
  assert(type(right) is str)

  # Trace print
  # print(f"{left:<8} {right:<}")

  if left in visited or right in visited:
    # To avoid cycles, abort if we've already visited a node
    # NOTE: Not like 100% sure this is clean
    print("WARNING! Cycle detected in the relations.")
    return None

  if left == right:
    return left
  
  for next_left in relations[left]:
    left_res = get_node_join(next_left, right, relations, visited=visited)
    if left_res is None:
      # When you reach the leaf of a spanning tree, pop back up to recurse
      for next_right in relations[right]:
        return get_node_join(left, next_right, relations, visited=visited)
    else:
      return left_res

  if len(relations[right]) == 0 and len(relations[left]) == 0:
    return None


In [18]:
print('vlines, hlines => ' + str(get_node_join('vlines', 'hlines', core_objs)))
print('rects, hlines => ' + str(get_node_join('rects', 'hlines', core_objs)))
print('points, hlines => ' + str(get_node_join('points', 'hlines', core_objs)))

vlines, hlines => lines
rects, hlines => regions
points, hlines => None


## Prepare structure relations
We want to be able to say that two structures have the same time if they both map to a common structure (if they have common structure eg. gui.stack) and structure type.

In [19]:
# def register_struct_relation(registry, )

## Find Overlaps
An overlap is a spec that is found in each side. There are usually a lot, and ideally we want the "maximal" overlaps (and not all of its the subsets).

The strategy is to find an overlap, exploit it as much as possible, then move on.

How do you know if you're done?  
-- You've looped through all things and stopped adding things.

When do you duplicate your candidate?  
-- I guess when you're going back up  

Are we going depth first or breadth first?  
-- Depth first makes the most sense I think  

How do you even recurse, since every node behaves differently?   
-- Well I guess you need a traversal function that takes a "deal with node" function with parameters for local data  

How do you traverse two specs at once?  
-- Probably like the relation meet algo: progress in one, and when it's done, pop back to the top before recursing

What if I look at _just_ the structures, then extend them with representations and behavior?  
-- That can work, as long as I can match representations on their own (ie. if no structure stuff maps)

How do I compare structures.affects without m*n valid matches?  
-- The greedy approach feels like reaching into the uses for those objects, and basically buidling the graph.
-- I think a smarter way to do it is to match all the structures/representations/behaviors based on type and used fields, then in a second pass flesh them out with objects. 

In [20]:
# Get object relation registry
def traverse_spec(spec, func):
  # Navigate Structures
  for struct in spec.get('structures', []):
    func(struct)
  # Hm.

class Context(enum.Enum):
  Structure = enum.auto()
  Representation = enum.auto()
  Behavior = enum.auto()

def add_to_overlap():
  pass

def compare_structs(sinister, dextera, struct_relations):
  # Check Type
  common_type = None
  if sinister.get('type') == dextera.get('type'):
    common_type = sinister.get('type')
  # TODO: find join under struct_relations
  
  # TODO: how do I compare these without m*n valid matches?
  # Find overlap on affects
  pass
  
  # Find overlap on overlaps
  pass

def compare_nodes(s_node, d_node, context, obj_relations, struct_relations):
  assert(type(context) is Context)

  if context == Context.Structure:
    compare_structs(s_node, d_node, struct_relations)
  elif context == Context.Representation:
    pass
  elif context == Context.Behavior:
    pass

def traverse_specs(sinister, dextera, node_func):
  pass

## Generic graph matching algorithm
Taken from: https://link.springer.com/chapter/10.1007/978-1-4615-4401-2_10
With help from Brian (Hemple)

Update: this is all kind of bad

In [21]:
from dataclasses import dataclass
from typing import List, Tuple, Set
from copy import copy

In [22]:
# Error-Tolerant Graph Matching
class NodeKind(enum.Enum):
  Object = enum.auto()
  ObjectType = enum.auto()
  Structure = enum.auto()
  StructureType = enum.auto()
  Representation = enum.auto()
  RepresentationType = enum.auto()
  Behavior = enum.auto()

class EdgeKind(enum.Enum):
  Type = enum.auto()

  # Structures
  Affects = enum.auto()
  CoveredBy = enum.auto()
  MapToStruct = enum.auto()
  Derivative = enum.auto()

  # Objects
  MapToObj = enum.auto()

@dataclass
class Node:
  id: int
  kind: NodeKind
  def __str__(self):
    return f'{self.kind}: {self.id}'

@dataclass
class Edge:
  id: int
  kind: EdgeKind
  source: int
  target: int

  def __str__(self):
    return f'{self.kind}: ({self.source}-{self.target})'

@dataclass
class Graph:
  nodes: List[Node]
  edges: List[Edge]

  def __str__(self):
    res = 'Nodes:\n'
    res += ', '.join([str(n) for n in self.nodes])
    res += '\n'
    res += 'Edges: \n'
    res += '\n'.join([str(e) for e in self.edges])
    return res

__next_id = 0
def new_id():
  global __next_id
  current = __next_id
  __next_id += 1
  return current

def reset_id_gen():
  global __next_id
  print("WARNING: Resetting ID Gen, any previously generated ids are no longer meaningful.")
  __next_id = 0

def make_graph(spec_file: str, verbose=False) -> Graph:
  def vprint(arg):
    if verbose:
      print(arg)

  graph = Graph([], [])
  
  assert(spec_file[-5:] == '.yaml')
  with open(spec_file, 'r') as file_handle:
    spec = yaml.safe_load(file_handle)

  # Objects
  obj_registry = register_spec_objects(spec)
  obj_ids: dict[str, int] = {} # {name: id}
  for key in obj_registry:
    o_id = new_id()
    obj_ids[key] = o_id
    graph.nodes.append(Node(id=o_id, kind=NodeKind.Object))
    # print(f'{key}: {obj_ids[key]}')

  for obj, targets in obj_registry.items():
    for target in targets:
      # assert(obj_ids.get(target) is not None)
      if obj_ids.get(target) is None:
        # TODO: this should be prevented by dealing with core components properly
        vprint('note: adding id for ' + target)
        obj_ids[target] = new_id()
      src = obj_ids.get(obj)
      tgt = obj_ids.get(target)
      assert(src is not None and tgt is not None)
      graph.edges.append(Edge(id=new_id(), kind=EdgeKind.MapToObj, source=src, target=tgt))
  # TODO: add edges that account for transitivity

  # Structures
  struct_ids: dict[str, int]= {} # {name: id}
  struct_type_ids: dict[str, int]= {} # {name: id}
  for struct in spec.get('structures', []):
    struct_name = struct.get('name')
    assert(struct_name is not None)

    struct_node = Node(new_id(), NodeKind.Structure)
    graph.nodes.append(struct_node)

    assert(struct_ids.get(struct_name) is None)
    struct_ids[struct_name] = struct_node.id
    vprint(f'{struct_name}: {struct_node.id}')

    # Handle structure type: struct-->struct_type
    if struct_type := struct.get('type', False):
      type_id = struct_type_ids.get(struct_type, new_id()) # create new id if it doesn't exist
      if not struct_type_ids.get(struct_type):
        struct_type_ids[struct_type] = type_id
        graph.nodes.append(Node(id=type_id, kind=NodeKind.StructureType)) # and add it to nodes
      
      graph.edges.append(Edge(id=new_id(), kind=EdgeKind.Type, source=struct_node.id, target=type_id))
    
    # Handle Affects: struct-->affected object
    if affected := struct.get('affects'):
      def handle_affected(obj):
        obj_id = obj_ids.get(obj)
        assert(obj_id is not None)
        graph.edges.append(Edge(id=new_id(), kind=EdgeKind.Affects, source=struct_node.id, target=obj_id))
      
      process_listable(affected, handle_affected)
    
    # Handle Covers: struct-->covering object
    if affected := struct.get('covered-by'):
      def handle_cover(obj):
        obj_id = obj_ids.get(obj)
        assert(obj_id is not None)
        graph.edges.append(Edge(id=new_id(), kind=EdgeKind.CoveredBy, source=struct_node.id, target=obj_id))
      
      process_listable(affected, handle_cover)
    
    # Handle mapto: struct-->other-struct
    if affected := struct.get('mapto'):
      def handle_mapto(target_struct):
        struct_id = struct_ids.get(target_struct)
        assert(struct_id is not None)
        graph.edges.append(Edge(id=new_id(), kind=EdgeKind.MapToStruct, source=struct_node.id, target=struct_id))
      
      process_listable(affected, handle_mapto)
    
    # TODO: Handle nested structures


  vprint('object ids:')
  vprint(obj_ids)
  vprint('structure ids:')
  vprint(struct_type_ids)
  vprint(graph)
  return graph
  


In [23]:
reset_id_gen()
make_graph('calendar.yaml', True)

note: adding id for hlines
note: adding id for rects
note: adding id for regions
note: adding id for icons
time: 23
object ids:
{'timestamps': 0, 'days': 1, 'weeks': 2, 'events': 3, 'days.selected': 4, 'timestamps.now': 5, 'days.selected->timestamps': 6, 'days.selected->events': 7, 'day-view': 8, 'nav-week': 9, 'hlines': 14, 'rects': 17, 'regions': 19, 'icons': 21}
structure ids:
{'linear': 24}
Nodes:
NodeKind.Object: 0, NodeKind.Object: 1, NodeKind.Object: 2, NodeKind.Object: 3, NodeKind.Object: 4, NodeKind.Object: 5, NodeKind.Object: 6, NodeKind.Object: 7, NodeKind.Object: 8, NodeKind.Object: 9, NodeKind.Structure: 23, NodeKind.StructureType: 24
Edges: 
EdgeKind.MapToObj: (4-1)
EdgeKind.MapToObj: (4-0)
EdgeKind.MapToObj: (4-3)
EdgeKind.MapToObj: (5-0)
EdgeKind.MapToObj: (5-14)
EdgeKind.MapToObj: (6-14)
EdgeKind.MapToObj: (7-17)
EdgeKind.MapToObj: (8-19)
EdgeKind.MapToObj: (9-21)
EdgeKind.Type: (23-24)
EdgeKind.Affects: (23-0)
EdgeKind.CoveredBy: (23-1)
EdgeKind.CoveredBy: (23-2)
Edge

Graph(nodes=[Node(id=0, kind=<NodeKind.Object: 1>), Node(id=1, kind=<NodeKind.Object: 1>), Node(id=2, kind=<NodeKind.Object: 1>), Node(id=3, kind=<NodeKind.Object: 1>), Node(id=4, kind=<NodeKind.Object: 1>), Node(id=5, kind=<NodeKind.Object: 1>), Node(id=6, kind=<NodeKind.Object: 1>), Node(id=7, kind=<NodeKind.Object: 1>), Node(id=8, kind=<NodeKind.Object: 1>), Node(id=9, kind=<NodeKind.Object: 1>), Node(id=23, kind=<NodeKind.Structure: 3>), Node(id=24, kind=<NodeKind.StructureType: 4>)], edges=[Edge(id=10, kind=<EdgeKind.MapToObj: 6>, source=4, target=1), Edge(id=11, kind=<EdgeKind.MapToObj: 6>, source=4, target=0), Edge(id=12, kind=<EdgeKind.MapToObj: 6>, source=4, target=3), Edge(id=13, kind=<EdgeKind.MapToObj: 6>, source=5, target=0), Edge(id=15, kind=<EdgeKind.MapToObj: 6>, source=5, target=14), Edge(id=16, kind=<EdgeKind.MapToObj: 6>, source=6, target=14), Edge(id=18, kind=<EdgeKind.MapToObj: 6>, source=7, target=17), Edge(id=20, kind=<EdgeKind.MapToObj: 6>, source=8, target=19),

In [24]:
make_graph('video-editor.yaml')

Graph(nodes=[Node(id=30, kind=<NodeKind.Object: 1>), Node(id=31, kind=<NodeKind.Object: 1>), Node(id=32, kind=<NodeKind.Object: 1>), Node(id=33, kind=<NodeKind.Object: 1>), Node(id=34, kind=<NodeKind.Object: 1>), Node(id=35, kind=<NodeKind.Object: 1>), Node(id=36, kind=<NodeKind.Object: 1>), Node(id=37, kind=<NodeKind.Object: 1>), Node(id=50, kind=<NodeKind.Structure: 3>), Node(id=51, kind=<NodeKind.StructureType: 4>), Node(id=56, kind=<NodeKind.Structure: 3>), Node(id=57, kind=<NodeKind.StructureType: 4>), Node(id=60, kind=<NodeKind.Structure: 3>)], edges=[Edge(id=39, kind=<EdgeKind.MapToObj: 6>, source=31, target=38), Edge(id=40, kind=<EdgeKind.MapToObj: 6>, source=31, target=30), Edge(id=42, kind=<EdgeKind.MapToObj: 6>, source=32, target=41), Edge(id=43, kind=<EdgeKind.MapToObj: 6>, source=33, target=41), Edge(id=44, kind=<EdgeKind.MapToObj: 6>, source=33, target=35), Edge(id=46, kind=<EdgeKind.MapToObj: 6>, source=34, target=45), Edge(id=47, kind=<EdgeKind.MapToObj: 6>, source=35, 

In [25]:
# Python type for a set of node pairs
ETGM = Set[Tuple[int, int]] # Error Tolerant Graph Matching

def score_etgm(g1: Graph, g2: Graph, etgm: ETGM) -> float:
  # TODO ...
  # missing_nodes_g1 = ...
  # missing_nodes_g2 = ...
  # replace_costs = [replacement_cost(n1, n2) for (n1, n2) in etgm]
  # edge_costs = 0.0
  # for (n1, n2) in etgm:
  #   node_missing_edges_g1 = ...
  #   node_missing_edges_g2 = ...
  #   ...
  return 0.0

def etgms_scored_sorted(g1: Graph, g2: Graph) -> List[Tuple[ETGM, float]]:
  return sorted([(etgm, score_etgm(g1, g2, etgm)) for etgm in get_etgms(g1, g2)], key=lambda x: x[1])

def get_etgms(g1: Graph, g2: Graph) -> List[ETGM]:
  cache = {}

  def expand_etgms(etgms: List[ETGM]) -> Tuple[bool, List[ETGM]]:
    expanded = False
    out_etgms = copy(etgms)
    for etgm in etgms:
      expanded_etgm = devandvscodescoolfunction(etgm)
      if expanded_etgm is not None and expanded_etgm not in out_etgms:
        out_etgms.append(expanded_etgm)
        expanded = True
    # print(out_etgms)
    return expanded, out_etgms

  def devandvscodescoolfunction(etgm):
      if str(etgm) in cache:
        return cache[str(etgm)]
      
      for n1 in g1.nodes:
        for n2 in g2.nodes:
          if (n1.id, n2.id) not in etgm:
            if is_valid_node_pair(n1, n2):
              new_etgm = set(etgm) #??
              new_etgm.add((n1.id, n2.id))
              # print((n1.id, n2.id), etgms)
              if is_valid_etgm(g1, g2, new_etgm):
              # expanded = True
                cache[str(etgm)] = set(new_etgm)
                return new_etgm
              # print(new_etgm, etgms)
              # out_etgms.append(new_etgm)
      cache[str(etgm)] = None
      return None

  etgms = [set()]
  expanded = True
  while expanded:
    expanded, etgms = expand_etgms(etgms)
    print(len(etgms))
  return etgms

def is_valid_node_pair(n1: Node, n2: Node) -> bool:
  if n1.kind != n2.kind:
    return False
  
  return True

def is_valid_etgm(g1: Graph, g2: Graph, etgm: ETGM) -> bool:
  # for every node pair in etgm
  # for (n1, n2) in etgm:
  return True

In [26]:
g1 = make_graph('calendar.yaml')
g2 = make_graph('video-editor.yaml')
n = len(g1.nodes)
m = len(g2.nodes)
print(f'n*m*2^(n*m)={n*m}*2^{n*m}={n*m*2**(n*m):e}')

n*m*2^(n*m)=156*2^156=1.424964e+49


In [27]:
reset_id_gen()
# g1 = make_graph('test1.yaml')
# g2 = make_graph('test1.yaml')
g1 = make_graph('calendar.yaml')
g2 = make_graph('video-editor.yaml')
# get_etgms(g1, g2)

### ok what if i use itertools


In [28]:
s1 = set({1,2,3})
s2 = set({'a','b', 'c'})
pprint(set(itertools.product(s1, s2)))

s1 = set({1,2,3})
s2 = set({'a','b', 'c'})
pprint(list(itertools.product([1,2], ['a', 'b', 'c'])))

{(1, 'a'),
 (1, 'b'),
 (1, 'c'),
 (2, 'a'),
 (2, 'b'),
 (2, 'c'),
 (3, 'a'),
 (3, 'b'),
 (3, 'c')}
[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c')]


In [39]:
 
# v good and smart

# Returns
# [
#   [first pairing],
#   [second pairing],
#   ...
# ]
def list_pairings(s1: Set, s2: Set):
  assert(len(s1) <= len(s2))

  l1 = list(s1)
  mappings = []
  for subset_choice in itertools.combinations(s2, len(s1)):
    # Pick a subset that you want to look at
    for permutation in itertools.permutations(subset_choice):
      # try all permutations of s2 against (static) s1
      mappings.append(list(zip(l1, permutation)))
  
  return mappings

def count_list_pairings(s1: Set, s2: Set):
  assert(len(s1) <= len(s2))
  return math.comb(len(s2), len(s1)) * math.perm(len(s1))


list_pairings(s1, s1)


[[(1, 1), (2, 2), (3, 3)],
 [(1, 1), (2, 3), (3, 2)],
 [(1, 2), (2, 1), (3, 3)],
 [(1, 2), (2, 3), (3, 1)],
 [(1, 3), (2, 1), (3, 2)],
 [(1, 3), (2, 2), (3, 1)]]

In [51]:
s3 = set(range(4))
print(len(list_pairings(s3, s3)))
print(count_list_pairings(s3, s3))


24
24


In [71]:
def subset_pairings(s1: Set, s2: Set, size):
  l1 = list(s1)
  l2 = list(s2)

  assert(size <= len(l1) <= len(l2))

  res = []
  for subset1 in itertools.combinations(s1, size):
    for subset2 in itertools.combinations(s2, size):
      # Pick out a subset for s1 and s2, and list their pairings
      # print(len(list_pairings(subset1, subset2)))
      res += list_pairings(subset1, subset2)

  return res

def count_subset_pairings(s1, s2, size: int):
  assert(size <= len(s1) <= len(s2))
  new_s1_size = len(s1) - size
  new_s2_size = len(s2) - size
  print('new size', new_s1_size)
  n1 = list(range(new_s1_size))
  n2 = list(range(new_s2_size))
  print(math.comb(len(s1), size))
  print(math.comb(len(s2), size))
  print(count_list_pairings(n1, n2))
  print('TODO: I think this is wrong')
  # return math.comb(len(s1), size) * math.comb(len(s2), size) * count_list_pairings(n1, n2)

subset_pairings(s1, s2, 0)

[[]]

In [70]:
# make a set 
sz = 2
print('input size: ', len(s1))
print('res = ' + str(len(subset_pairings(s1, s1, sz))))
print('res = ' + str(count_subset_pairings(s1, s1, sz)))

input size:  3
2
2
2
2
2
2
2
2
2
res = 18
new size 1
3
3
1
TODO: I think this is wrong
res = None


In [25]:
def all_subset_pairings(s1: Set, s2: Set, verbose=False):
  acc = []
  for i in range(min(len(s1), len(s2))):
    size = i + 1
    inter_res = subset_pairings(s1, s2, size)
    acc += inter_res

    if verbose:
      num_res = len(inter_res)
      print(f'size = {size}')
      print(inter_res)
      print(f'len = {num_res}')
      print()
  return acc

print('total: ' + str(len(all_subset_pairings(s1, s2, verbose=True))))


size = 1
[[(1, 'a')], [(1, 'b')], [(1, 'c')], [(2, 'a')], [(2, 'b')], [(2, 'c')], [(3, 'a')], [(3, 'b')], [(3, 'c')]]
len = 9

size = 2
[[(1, 'a'), (2, 'b')], [(1, 'b'), (2, 'a')], [(1, 'a'), (2, 'c')], [(1, 'c'), (2, 'a')], [(1, 'b'), (2, 'c')], [(1, 'c'), (2, 'b')], [(1, 'a'), (3, 'b')], [(1, 'b'), (3, 'a')], [(1, 'a'), (3, 'c')], [(1, 'c'), (3, 'a')], [(1, 'b'), (3, 'c')], [(1, 'c'), (3, 'b')], [(2, 'a'), (3, 'b')], [(2, 'b'), (3, 'a')], [(2, 'a'), (3, 'c')], [(2, 'c'), (3, 'a')], [(2, 'b'), (3, 'c')], [(2, 'c'), (3, 'b')]]
len = 18

size = 3
[[(1, 'a'), (2, 'b'), (3, 'c')], [(1, 'a'), (2, 'c'), (3, 'b')], [(1, 'b'), (2, 'a'), (3, 'c')], [(1, 'b'), (2, 'c'), (3, 'a')], [(1, 'c'), (2, 'a'), (3, 'b')], [(1, 'c'), (2, 'b'), (3, 'a')]]
len = 6

total: 33


In [ ]:
def count_all_subset_pairings(s1, s2, verbose=False):
  count = 0
  max_size = min(len(s1), len(s2))
  if verbose:
    print(f'Going up to size {max_size}')
  for i in range(max_size):
    size = i + 1
    inter_res = len(subset_pairings(s1, s2, size))
    count += inter_res

    if verbose:
      print(f'size = {size}')
      print(f'len = {inter_res}')
      print()
  return count

count_all_subset_pairings(g1.nodes, g2.nodes, True)

Going up to size 12
size = 1
len = 156

size = 2
len = 10296

size = 3
len = 377520

size = 4
len = 8494200



In [ ]:
all_subset_pairings(g1.nodes, g2.nodes)

In [ ]:

# def direct_pairings(s1: Set, s2: Set, nest=0) -> List[List[Tuple[int, int]]]:
#   sp = ' '*2*nest

#   if len(s1) == 1:
#     res = []
#     x = list(s1)[0]
#     for y in s2:
#       res.append((x, y))

#     print(sp + str([res]))
#     return [res]

#   res = []
#   for x in s1:
#     for y in s2:
#       print(sp + f'pin ({x}, {y})')
#       rec = direct_pairings(s1 - set({x}), s2 - set({y}), nest+1)
#       # print(sp + str(rec))
#       for new_mapping in rec:
#         res.append([(x, y)] + new_mapping)
#       # print(res[-1])
#       print(sp+str(res))
#       print()
#     # print(sp + str(res))
#   return res
# ^ bad and dumb

def len_subset_pairings(m: int, n: int, size=None) -> int:
  if size is None:
    size = min(m, n)
  elif size == 0:
    return 1
  
  return len_subset_pairings(m, n, size - 1) * (m-size+1) * (n-size+1)

# def subset_pairings(s1: Set[int], s2: Set[int], size: int) -> List[Tuple[int, int]]:
#   # enforce size <= len(m) <= len(n)
#   if len(s1) <= len(s2):
#     m, n = s1, s2
#   else:
#     m, n = s2, s1
#   assert(size <= len(m))
  
#   # for each element of common subset of len=size...
#   for i in range(size):
#     # pick an item from s1...
    

In [63]:
len_subset_pairings(len(s1), len(s2))

3
2
1


36

In [68]:
res = len_subset_pairings(len(g1.nodes), len(g2.nodes))
print(f'{res:e}')

12
11
10
9
8
7
6
5
4
3
2
1
2.982753e+18


# Ok new plan
- construct graph with only structures as vertices
  - actually use the type, which will be used when rating/ranking
- add edges between structures 
  - if they have a relation (mapto, contrain/derive)
  - if they both point to the same object, add an edge between them labeled with the object join